# Using PyMC

[PyMC](https://www.pymc.io) is a very powerful Python library designed for probabilistic and Bayesian analysis. 
Here, we show that PyMC can be used to perform the same likelihood sampling that we previously wrote our own algorithm for. 

Below, we read in the data and build the model.

In [1]:
import pandas as pd 
import numpy as np
from scipy.stats import norm

data = pd.read_csv('../data/first-order.csv')

D = [norm(data['At'][i], data['At_err'][i]) for i in range(len(data))]

def first_order(t, k, A0):
    """
    A first order rate equation.
    
    :param t: The time to evaluate the rate equation at.
    :param k: The rate constant.
    :param A0: The initial concentration of A.
    
    :return: The concentration of A at time t.
    """
    return A0 * np.exp(-k * t)

The next step is to construct the PyMC sampler. 
The format that PyMC expects can be a bit unfamiliar. 

First we create objects for the two parameters, these are bounded so $0 \leq k < 1$ and $0 \leq [A]_0 < 10$.
Strictly, these are [prior probabilities](./priors.md), which we will look at next, but using uniform distributions means this is mathematically equivalent to likelihood sampling.
Next, we create a normally distributed likelihood function to compare the data and the model. 
Finally, we sample for 1000 steps, with 10 chains. 
The `tune` parameter is the number of steps for tuning the Markov chain step sizes. 
````{margin}
```{note}
Other parameters for the `pm.sample` method can be found in the [appropriate documentation](https://www.pymc.io/projects/docs/en/stable/api/generated/pymc.sample.html).
```
````

In [ ]:
import pymc as pm

with pm.Model() as model:
    k = pm.Uniform('k', 0, 1)
    A0 = pm.Uniform('A0', 0, 10)
    
    At = pm.Normal('At', 
                   mu=first_order(data['t'], k, A0), 
                   sigma=data['At_err'], 
                   observed=data['At'])
    
    trace = pm.sample(1000, tune=1000, chains=10, progressbar=True)

Unlike the code that we created previously, PyMC defaults to using the NUTS sampler, which stands for No-U-Turn sampler {cite}`Hoffman2014`. 
This sampler enables the step size tuning that we have taken advantage of. 

This results in a object assigned to the variable `trace`. 

In [ ]:
trace

This contains the chain information amoung other things. 
Instead of probing into the `trace` object, we can take advantage of functionality from the `arviz` library to produce some informative plots. 

In [ ]:
import matplotlib.pyplot as plt
import arviz as az

az.plot_trace(trace, var_names=["k", "A0"])
plt.tight_layout()
plt.show()

Above, we can see the trace of each of the different chains. 
The chains appear to have converged to the same distribution.
We can get the flat chains with the following function.

In [ ]:
flat_chain = np.vstack([trace.posterior['k'].values.flatten(), trace.posterior['A0'].values.flatten()]).T

import seaborn as sns

chains_df = pd.DataFrame(flat_chain, columns=['k', 'A0'])
sns.jointplot(data=chains_df, x='k', y='A0', kind='kde')
plt.show()

It is clear that, using PyMC, we have much better sampling of the distributions. 
This makes using summary statistics, like the mean and standard deviation much more reliable. 

In [ ]:
az.summary(trace, var_names=["k", "A0"])